<a href="https://colab.research.google.com/github/nataliapuentesca-crypto/sistemas-avanzados-de-produccion/blob/main/NataliaPuentes_DennisCarrillo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taller 3: Regresión Lineal y Conceptos Fundamentales



## Instrucciones Generales

**Fecha de entrega:** 20 de septiembre.
**Tiempo disponible:** Tienen dos clases y el fin de semana para completar el taller.

1.  **Trabajo en Grupo:** Este taller se debe realizar en los grupos previamente definidos.
2.  **Repositorio en GitHub:** Cada grupo debe crear un repositorio en GitHub.
    * Una sola persona del grupo es responsable de crearlo.
    * El repositorio debe llamarse **`Regresion_Lineal`**.
    * Este archivo debe ser subido al repositorio con el nombre **`Regresion_Lineal.ipynb`**.
    * **Alternativa:** Si tienen complicaciones con GitHub, pueden enviar el enlace del cuaderno de Google Colab al aula virtual.
3.  **Actividad a Mano (20% de la nota):**
    * La asistencia a clase es **obligatoria**.
    * Se realizará una actividad práctica de regresión lineal a mano.
    * Cada integrante del grupo deberá firmar la hoja de la actividad, la cual constituye el 20% de la nota de este taller.
    

## **Parte 1: Definición del Problema**

Piensa en tu experiencia académica o laboral en un área de operaciones y elige **un problema** relacionado con inventarios, calidad, mantenimiento o productividad.

1.  Indica cuál es tu variable dependiente **$Y$** (aquello que quieres predecir o clasificar).
2.  Lista entre 3 y 5 variables independientes **$X$** que medirías para predecir $Y$.
3.  Especifica si tu problema es de **regresión** (predecir un valor numérico) o de **clasificación**.

*Problema Seleccionado: Predicción de Defectos en Línea de Producción
Contexto:
En una planta de manufactura de componentes electrónicos, es crítico predecir cuándo los productos tendrán defectos para optimizar el control de calidad y reducir costos de reproceso.

1. Variable Dependiente (Y):
Y = Tasa de Defectos por Lote

Definición: Porcentaje de productos defectuosos en cada lote de producción
Unidad: Porcentaje (0% - 100%)
Ejemplo: Si un lote de 1,000 unidades tiene 25 defectuosas, Y = 2.5%


2. Variables Independientes (X):
X₁ = Temperatura del Proceso de Soldadura

Unidad: Grados Celsius (°C)
Rango típico: 220°C - 260°C
Justificación: Temperaturas fuera del rango óptimo afectan la calidad de las soldaduras

X₂ = Humedad Relativa del Ambiente

Unidad: Porcentaje (%)
Rango típico: 40% - 65%
Justificación: La humedad excesiva puede causar oxidación y problemas en componentes sensibles

X₃ = Velocidad de la Línea de Producción

Unidad: Unidades por hora (uph)
Rango típico: 800 - 1,200 uph
Justificación: Velocidades muy altas pueden comprometer la calidad por tiempo insuficiente de proceso

X₄ = Tiempo desde el Último Mantenimiento Preventivo

Unidad: Horas de operación
Rango típico: 0 - 200 horas
Justificación: Equipos con más tiempo sin mantenimiento tienden a generar más defectos

X₅ = Experiencia del Operador del Turno

Unidad: Años de experiencia
Rango típico: 0.5 - 15 años
Justificación: Operadores más experimentados detectan y corrigen problemas más rápidamente


3. Tipo de Problema: REGRESIÓN
Justificación:

Variable dependiente numérica continua: La tasa de defectos es un porcentaje que puede tomar cualquier valor entre 0% y 100%
Objetivo de predicción: Queremos predecir el valor específico de la tasa de defectos
Interpretación: Un modelo de regresión nos permitirá predecir numéricamente qué tan alta será la tasa de defectos bajo diferentes condiciones

## **Parte 2: Preprocesamiento de Datos y *Leakage***

Basado en el caso que definiste en la Parte 1:

1.  Lista entre 3 y 5 **transformaciones** que aplicarías a tus datos (ej. imputación de valores faltantes, codificación de variables categóricas, escalado, creación de *lags*, etc.) y **justifica por qué** cada una es necesaria.
2.  Señala un posible riesgo de ***data leakage*** (fuga de datos) en tu plan y explica cómo lo evitarías usando un *pipeline* de preprocesamiento.


. Transformaciones de Datos
Transformación 1: Imputación de Valores Faltantes
Método: Imputación por mediana para variables numéricas y forward-fill para datos temporales
Variables afectadas: Todas las X (especialmente temperatura y humedad por fallos de sensores)
Justificación:

Realidad operativa: Los sensores de temperatura y humedad pueden fallar temporalmente o generar lecturas erróneas
Por qué mediana: Menos sensible a outliers que la media, común en datos de manufactura
Forward-fill: Para datos temporales, el último valor válido es una aproximación razonable para períodos cortos

In [2]:
# Para valores faltantes esporádicos
# Assuming 'temperatura' is a column in your 'data' DataFrame
data['temperatura_filled'] = data['temperatura'].fillna(data['temperatura'].median())

# Para secuencias temporales cortas
# Assuming 'humedad' is a column in your 'data' DataFrame
data['humedad_filled'] = data['humedad'].fillna(method='ffill', limit=3)

NameError: name 'data' is not defined

Transformación 2: Detección y Tratamiento de Outliers
Método: Método IQR (Rango Intercuartílico) con winsorización
Variables afectadas: Temperatura, velocidad de línea, humedad
Justificación:

Outliers comunes: Temperaturas extremas por mal funcionamiento, velocidades anómalas por paradas de emergencia
Impacto en modelos: Los outliers pueden sesgar significativamente modelos de regresión
Winsorización vs eliminación: Preservamos el tamaño del dataset reemplazando valores extremos por percentiles

In [3]:
# Detectar outliers con IQR
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
IQR = Q3 - Q1
# Winsorizar a percentiles 5-95
data_clean = data.clip(lower=data.quantile(0.05), upper=data.quantile(0.95))

NameError: name 'data' is not defined

Transformación 3: Escalado/Normalización (StandardScaler)
Método: Estandarización Z-score
Variables afectadas: Temperatura (°C), Humedad (%), Velocidad (uph), Tiempo mantenimiento (horas)
Justificación:

Diferentes escalas: Temperatura (~240°C), Humedad (~50%), Velocidad (~1000 uph)
Algoritmos sensibles: Regresión lineal, SVM, redes neuronales requieren variables en escala similar
Interpretación de coeficientes: Permite comparar la importancia relativa de variables

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_numeric)

Transformación 4: Creación de Variables de Interacción y Lags
Variables nuevas:

Lag de temperatura: Temperatura del lote anterior
Interacción temperatura-humedad: Producto de temperatura normalizada × humedad normalizada
Ventana móvil de defectos: Promedio de defectos de los últimos 3 lotes

Justificación:

Memoria del proceso: Los defectos actuales pueden depender de condiciones previas
Efectos combinados: Temperatura + humedad alta pueden tener efecto multiplicativo en defectos
Tendencias: El promedio móvil captura tendencias que un punto individual no muestra

In [ ]:
# Lag features
data['temp_lag1'] = data['temperatura'].shift(1)
# Interacciones
data['temp_humidity_interaction'] = data['temperatura'] * data['humedad']
# Ventana móvil
data['defects_rolling_3'] = data['tasa_defectos'].shift(1).rolling(3).mean()

Transformación 5: Codificación de Variable Categórica (One-Hot Encoding)
Variable: Turno de trabajo (Mañana, Tarde, Noche)
Método: One-Hot Encoding
Justificación:

Variable categórica ordinal: Diferentes turnos pueden tener diferentes niveles de defectos
No asumir orden: One-hot evita asumir que "Noche" > "Tarde" > "Mañana"
Flexibilidad del modelo: Permite que cada turno tenga su propio coeficiente independiente

In [4]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first', sparse=False)
turno_encoded = encoder.fit_transform(data[['turno']])

TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'

2. Riesgo de Data Leakage y Prevención
Riesgo Identificado: Leakage Temporal
Problema específico: Usar información futura para predecir el pasado en la creación de ventanas móviles
Ejemplo del error:

In [ ]:
# ❌ INCORRECTO - Data Leakage
data['defects_rolling_3'] = data['tasa_defectos'].rolling(3).mean()
# Esto usa información del presente para "predecir" el presente

Por qué es problemático:

En producción: No tendremos la tasa de defectos actual para predecir la tasa de defectos actual
Overfitting artificial: El modelo parece mejor de lo que realmente es
Falla en implementación: El modelo no funcionará en producción real

Solución: Pipeline de Preprocesamiento Robusto

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
import pandas as pd

class SafeTimeSeriesTransformer:
    def __init__(self):
        self.scalers = {}

    def fit(self, X, y=None):
        # Solo usar datos históricos para calcular estadísticas
        for col in X.columns:
            if col.endswith('_lag') or col == 'temp_humidity_interaction':
                continue
            self.scalers[col] = StandardScaler()
            self.scalers[col].fit(X[col].values.reshape(-1, 1))
        return self

    def transform(self, X):
        X_transformed = X.copy()

        # ✅ CORRECTO - Solo usar información pasada
        X_transformed['temp_lag1'] = X['temperatura'].shift(1)
        X_transformed['defects_rolling_3'] = X['tasa_defectos'].shift(1).rolling(3).mean()
        X_transformed['temp_humidity_interaction'] = X['temperatura'] * X['humedad']

        # Escalar usando estadísticas del conjunto de entrenamiento
        for col, scaler in self.scalers.items():
            if col in X_transformed.columns:
                X_transformed[col] = scaler.transform(X_transformed[col].values.reshape(-1, 1)).flatten()

        return X_transformed

# Pipeline completo
preprocessing_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('time_features', SafeTimeSeriesTransformer()),
    ('scaler', StandardScaler())
])

# División temporal correcta
def temporal_split(data, train_ratio=0.7):
    split_point = int(len(data) * train_ratio)
    return data[:split_point], data[split_point:]

# Uso correcto
train_data, test_data = temporal_split(data)

# Fit solo en datos de entrenamiento
preprocessing_pipeline.fit(train_data.drop('tasa_defectos', axis=1))

# Transform por separado
X_train_processed = preprocessing_pipeline.transform(train_data.drop('tasa_defectos', axis=1))
X_test_processed = preprocessing_pipeline.transform(test_data.drop('tasa_defectos', axis=1))

Principios Clave para Evitar Leakage:

División temporal estricta: Entrenar solo con datos anteriores al período de prueba
Estadísticas solo del entrenamiento: Medias, desviaciones, etc., calculadas solo en train
Lags apropiados: Usar siempre .shift(1) o más para variables target
Validación temporal: Cross-validation que respete el orden temporal
Pipeline consistente: Mismas transformaciones en entrenamiento y producción

Validación del pipeline:

In [ ]:
# Verificar que no hay leakage
assert X_train_processed.shape[0] == train_data.shape[0]
assert not X_train_processed.isnull().any().any()  # Sin valores faltantes
assert X_test_processed.mean().abs().max() < 3     # Escalado correcto

Este enfoque garantiza que el modelo entrenado será replicable en producción, evitando la trampa común del data leakage temporal que puede hacer que un modelo parezca excelente en desarrollo pero falle completamente en implementación real.

## **Parte 3: Interpretación y Métricas de Regresión Simple**

Para esta sección, elige un caso simple de regresión (puede ser el tuyo o uno hipotético, como predecir la demanda de un producto según su precio).

1.  Define claramente las variables **$Y$** y **$X$** junto con sus **unidades** (ej. $Y$: número de unidades vendidas, $X$: precio en dólares).
1. Definición de Variables
Variable Dependiente (Y):

Y = Unidades vendidas por semana
Unidad: Número de unidades (entero positivo)
Rango típico: 50 - 500 unidades

Variable Independiente (X):

X = Precio de venta por unidad
Unidad: Dólares estadounidenses ($USD)
Rango típico: $15 - $45 por unidad

Contexto: Una tienda de electrónicos quiere predecir cuántas unidades de auriculares bluetooth venderá semanalmente según el precio que establezca.
2.  Supón que entrenas un modelo y obtienes una pendiente de **$\hat{\beta}_1 = -0.6$**. Escribe una interpretación clara y concisa de este coeficiente en el contexto de tu problema.
Modelo resultante:
Unidades Vendidas = β₀ + (-0.6) × Precio
Interpretación:

"Por cada dólar adicional que aumentemos el precio, esperamos vender 0.6 unidades menos por semana, manteniendo todo lo demás constante."

Ejemplos prácticos:

Si el precio actual es $25 y lo subimos a $30 (+$5), esperamos vender 3 unidades menos (5 × 0.6 = 3)
Si bajamos el precio de $35 a $30 (-$5), esperamos vender 3 unidades más

Significado económico:

El coeficiente negativo confirma la ley básica de la demanda (precio ↑ → demanda ↓)
La magnitud (-0.6) indica una sensibilidad moderada al precio
Es una relación inelástica en términos unitarios (< 1 en valor absoluto)
3.  ¿Qué **métrica** de evaluación usarías (MAE, RMSE, o MAPE) y **por qué** es la más adecuada para tu caso?
Fórmula:
MAPE = (1/n) × Σ |((Y_real - Y_predicho) / Y_real)| × 100
¿Por qué MAPE es la más adecuada?
1. Interpretabilidad para el negocio:

MAPE = 15% significa que nuestras predicciones se desvían en promedio 15% del valor real
Los gerentes entienden fácilmente los porcentajes vs. unidades absolutas

2. Escala independiente:

Permite comparar modelos entre diferentes productos con volúmenes distintos
Un error de 10 unidades es muy diferente si vendemos 50 vs. 500 unidades

3. Relevancia comercial:

En retail, los errores porcentuales son más relevantes para decisiones de inventario
Un error del 20% tiene implicaciones similares independientemente del volumen base

Comparación con otras métricas:
MAE (Mean Absolute Error):

Ventaja: Fácil interpretación en unidades originales
Desventaja: No considera la escala relativa (error de 10 unidades es diferente para productos de alta vs. baja rotación)

RMSE (Root Mean Square Error):

Ventaja: Penaliza más los errores grandes
Desventaja: Menos interpretable para stakeholders no técnicos, sensible a outliers
4.  Menciona **un supuesto** del modelo de regresión lineal que validarías (ej. linealidad, homocedasticidad) y explica **cómo** lo harías (usando un gráfico o una prueba estadística).
Supuesto Seleccionado: Homocedasticidad
Definición: La variabilidad de los residuos debe ser constante a lo largo de todos los valores predichos.
¿Por qué es crítico en este caso?

Si la variabilidad cambia con el precio, nuestras predicciones serán menos confiables en ciertos rangos
Violación común: Mayor variabilidad en ventas cuando los precios son muy bajos o muy altos

Métodos de Validación:
Método 1: Gráfico de Residuos vs. Valores Predichos

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Calcular residuos
residuos = y_real - y_predicho

# Crear gráfico
plt.figure(figsize=(10, 6))
plt.scatter(y_predicho, residuos, alpha=0.6)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('Valores Predichos (Unidades)')
plt.ylabel('Residuos')
plt.title('Residuos vs. Valores Predichos - Verificación de Homocedasticidad')
plt.grid(True, alpha=0.3)
plt.show()

Interpretación visual:

Homocedasticidad: Puntos distribuidos aleatoriamente alrededor de y=0 con varianza constante
Heterocedasticidad: Patrón de embudo (varianza creciente/decreciente)

In [ ]:
from statsmodels.stats.diagnostic import het_breuschpagan

# Realizar prueba
lm, lm_pvalue, fvalue, f_pvalue = het_breuschpagan(residuos, X_design_matrix)

print(f"Estadístico LM: {lm:.4f}")
print(f"P-valor: {lm_pvalue:.4f}")

# Interpretación
if lm_pvalue < 0.05:
    print("Rechazamos H₀: Evidencia de heterocedasticidad")
else:
    print("No rechazamos H₀: No hay evidencia suficiente de heterocedasticidad")

## **Parte 4: Regresión Múltiple y Colinealidad**

Volviendo a tu caso de la Parte 1 (con múltiples variables).

1.  Escribe el **vector de variables** $\vec{X}$ y la **respuesta** $Y$.
2.  Explica cómo interpretarías el **coeficiente** de una de tus variables clave (incluyendo unidades y el sentido de la relación: positiva o negativa).
3.  Si sospecharas que existe **colinealidad** entre tus variables, menciona **dos acciones** que podrías tomar para mitigarla.

1. Vector de Variables y RespuestaVector X⃗:
X⃗ = [X₁, X₂, X₃, X₄, X₅]Donde:

X₁ = Temperatura del proceso de soldadura (°C)
X₂ = Humedad relativa del ambiente (%)
X₃ = Velocidad de la línea de producción (unidades/hora)
X₄ = Tiempo desde último mantenimiento preventivo (horas)
X₅ = Experiencia del operador del turno (años)
Variable Respuesta Y:
Y = Tasa de defectos por lote (%)Modelo de Regresión Múltiple:
Y = β₀ + β₁X₁ + β₂X₂ + β₃X₃ + β₄X₄ + β₅X₅ + ε2. Interpretación del Coeficiente de Variable ClaveVariable seleccionada: X₄ (Tiempo desde último mantenimiento)Supongamos que obtenemos: β₄ = +0.025Interpretación completa:
"Por cada hora adicional que transcurre desde el último mantenimiento preventivo, esperamos que la tasa de defectos aumente en 0.025 puntos porcentuales, manteniendo constantes la temperatura, humedad, velocidad de línea y experiencia del operador."
Desglose detallado:Unidades:

Cambio en X₄: +1 hora
Cambio esperado en Y: +0.025 puntos porcentuales
Ratio de cambio: 0.025 puntos porcentuales por hora
Sentido de la relación:

Positiva: β₄ = +0.025 > 0
Lógica: A medida que pasa más tiempo sin mantenimiento, los equipos se deterioran y generan más defectos
Ejemplos prácticos:
Escenario base: Mantenimiento recién realizado (X₄ = 0 horas)

Tasa de defectos predicha: Y_base



Después de 40 horas: X₄ = 40 horas

Incremento esperado: 40 × 0.025 = 1.0 punto porcentual
Nueva tasa predicha: Y_base + 1.0%



Después de 100 horas: X₄ = 100 horas

Incremento esperado: 100 × 0.025 = 2.5 puntos porcentuales
Nueva tasa predicha: Y_base + 2.5%


Significado operacional:

Riesgo gradual: Los defectos aumentan progresivamente, no abruptamente
Punto de intervención: Si el coeficiente es significativo, ayuda a determinar intervalos óptimos de mantenimiento
Costo-beneficio: Balancear costo de mantenimiento vs. costo de defectos adicionales
3. Mitigación de ColinealidadColinealidad esperada en mi caso:Pares problemáticos potenciales:

X₁ y X₂: Temperatura y humedad pueden estar inversamente correlacionadas (aire caliente retiene menos humedad)
X₃ y X₄: Velocidad alta puede acelerar el desgaste, correlacionándose con tiempo de mantenimiento
X₄ y Y (variable latente): Tiempo de mantenimiento podría correlacionarse con vibraciones o desgaste no observado
Acción 1: Análisis de Matriz de Correlación y VIFDiagnóstico:

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Matriz de correlación
correlation_matrix = X_df.corr()
print("Correlaciones altas (|r| > 0.7):")
high_corr = np.where(np.abs(correlation_matrix) > 0.7)
for i, j in zip(*high_corr):
    if i != j:
        print(f"{X_df.columns[i]} - {X_df.columns[j]}: {correlation_matrix.iloc[i,j]:.3f}")

# Factor de Inflación de Varianza (VIF)
X_with_constant = sm.add_constant(X_df)
vif_data = pd.DataFrame()
vif_data["Variable"] = X_with_constant.columns[1:]  # Excluir constante
vif_data["VIF"] = [variance_inflation_factor(X_with_constant.values, i+1)
                   for i in range(len(X_with_constant.columns)-1)]

print("Variables con VIF > 5 (colinealidad problemática):")
print(vif_data[vif_data["VIF"] > 5])

Criterios de decisión:

VIF > 10: Colinealidad severa, acción requerida
VIF 5-10: Colinealidad moderada, monitorear
|r| > 0.8: Correlación alta entre variables

Solución específica:
Si encontramos X₁ (Temperatura) y X₂ (Humedad) altamente correlacionadas:

In [ ]:
# Eliminar la variable menos importante teóricamente
X_reduced = X_df.drop(['humedad'], axis=1)
# O crear una variable combinada
X_df['indice_comfort_termico'] = X_df['temperatura'] / X_df['humedad']

Acción 2: Regularización Ridge
Implementación:

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

# Definir rango de valores alpha para regularización
alpha_range = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

# Grid search para encontrar alpha óptimo
ridge_model = Ridge()
param_grid = {'alpha': alpha_range}

grid_search = GridSearchCV(
    ridge_model,
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    return_train_score=True
)

grid_search.fit(X_scaled, y)
best_alpha = grid_search.best_params_['alpha']

print(f"Alpha óptimo: {best_alpha}")

# Modelo final con regularización
ridge_final = Ridge(alpha=best_alpha)
ridge_final.fit(X_scaled, y)

Ventajas de Ridge en este contexto:
1. Contracción de coeficientes:

Reduce la magnitud de coeficientes correlacionados
Distribuye el "efecto" entre variables colineales

2. Estabilidad predictiva:

Reduce la varianza de las predicciones
Mejor generalización en datos nuevos

3. Mantiene todas las variables:

A diferencia de Lasso, no elimina variables completamente
Preserva interpretabilidad operacional

Comparación de coeficientes:

In [ ]:
# Comparar coeficientes antes y después de regularización
import matplotlib.pyplot as plt

coef_comparison = pd.DataFrame({
    'Variable': X_df.columns,
    'OLS': ols_model.coef_,
    'Ridge': ridge_final.coef_
})

coef_comparison.set_index('Variable').plot(kind='bar', figsize=(10, 6))
plt.title('Comparación de Coeficientes: OLS vs Ridge')
plt.ylabel('Valor del Coeficiente')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

Estrategia Integral:
1. Prevención durante recolección de datos:

Diseñar experimentos que minimicen correlaciones naturales
Recopilar datos en diferentes condiciones operativas

2. Monitoreo continuo:

Calcular VIF regularmente con nuevos datos
Establecer alertas automáticas cuando VIF > 5

3. Validación del impacto:

Comparar R² y métricas de error antes/después de mitigación
Verificar que la interpretabilidad se mantenga para stakeholders

4. Documentación operacional:

Registrar decisiones de eliminación/combinación de variables
Mantener trazabilidad para auditorías de calidad

Esta aproximación sistemática asegura que el modelo mantenga tanto precisión predictiva como interpretabilidad operacional, elementos críticos en un ambiente de manufactura donde las decisiones deben ser explicables y accionables.

## **Parte 5: Interacciones y Multicolinealidad (VIF)**

1.  Plantea un caso con una variable $Y$ y entre 4 y 6 variables $X$. ¿Qué término de **interacción** entre dos variables podrías añadir al modelo y **por qué** crees que sería útil?
2.  Si al calcular el Factor de Inflación de la Varianza (VIF) para una variable, obtienes un valor alto (ej. > 10), menciona **dos acciones** que podrías tomar para solucionarlo.

1. Definición del Modelo con InteracciónVariables del Modelo:Variable Dependiente:

Y = Puntuación de Satisfacción (escala 1-10)
Variables Independientes:

X₁ = Calidad de la comida (escala 1-10)
X₂ = Velocidad del servicio (minutos desde pedido hasta entrega)
X₃ = Precio promedio del plato (dólares)
X₄ = Ambiente del restaurante (escala 1-10: ruido, iluminación, decoración)
X₅ = Experiencia del mesero (años de experiencia)
X₆ = Día de la semana (1=Lunes, 7=Domingo)
Término de Interacción Propuesto: X₁ × X₂Modelo con interacción:
Y = β₀ + β₁X₁ + β₂X₂ + β₃X₃ + β₄X₄ + β₅X₅ + β₆X₆ + β₇(X₁ × X₂) + εDonde: X₁ × X₂ = Calidad de comida × Velocidad del servicio¿Por qué esta interacción es útil?Razón Conceptual:

"El efecto de la calidad de la comida en la satisfacción del cliente depende de qué tan rápido se sirve, y viceversa."
Escenarios prácticos:1. Comida excelente + Servicio rápido (X₁=9, X₂=15 min):

Efecto amplificado: La experiencia excepcional se potencia
Satisfacción esperada: Muy alta
2. Comida excelente + Servicio lento (X₁=9, X₂=45 min):

Efecto atenuado: La espera frustra al cliente pese a la buena comida
Satisfacción esperada: Moderada
3. Comida regular + Servicio rápido (X₁=6, X₂=15 min):

Compensación parcial: La rapidez compensa la calidad promedio
Satisfacción esperada: Aceptable
4. Comida regular + Servicio lento (X₁=6, X₂=45 min):

Efecto negativo combinado: Ambos aspectos decepcionantes
Satisfacción esperada: Baja
Interpretación del coeficiente β₇:Si β₇ = -0.008:

"Por cada minuto adicional de espera, el efecto positivo de la calidad de la comida en la satisfacción se reduce en 0.008 puntos por cada punto de calidad."
Ejemplo numérico:

Comida calidad 8, servicio 20 min: Efecto interacción = 8 × 20 × (-0.008) = -1.28 puntos
Comida calidad 8, servicio 40 min: Efecto interacción = 8 × 40 × (-0.008) = -2.56 puntos
Diferencia: 20 minutos extra de espera reducen 1.28 puntos adicionales la satisfacción
Valor para el negocio:

Estrategia operativa: Identificar el balance óptimo calidad-velocidad
Gestión de expectativas: En días ocupados, comunicar tiempos de espera
Capacitación de personal: Priorizar rapidez cuando la calidad sea consistente
Pricing dinámico: Ajustar precios según la combinación calidad-servicio ofrecida
2. Manejo de VIF Alto (VIF > 10)Ejemplo: Variable X₃ (Precio) con VIF = 15.2Este VIF alto indica que X₃ está altamente correlacionado con otras variables del modelo, lo que genera problemas de multicolinealidad.Acción 1: Eliminación Estratégica de VariablesProceso sistemático:Paso 1: Diagnosticar correlaciones específicas

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Identificar correlaciones altas
correlation_matrix = X_df.corr()
high_correlations = correlation_matrix['precio'].abs().sort_values(ascending=False)
print("Correlaciones de 'precio' con otras variables:")
print(high_correlations[high_correlations > 0.7])

# Ejemplo de resultado:
# ambiente         0.85  <- Restaurantes caros suelen tener mejor ambiente
# experiencia_mesero 0.78  <- Restaurantes caros contratan meseros experimentados

Paso 2: Decidir qué variable eliminar

In [ ]:
# Calcular VIF para todas las variables correlacionadas
variables_problema = ['precio', 'ambiente', 'experiencia_mesero']
vif_subset = []

for var in variables_problema:
    X_temp = X_df[variables_problema]
    X_temp = sm.add_constant(X_temp)
    vif = variance_inflation_factor(X_temp.values,
                                   variables_problema.index(var) + 1)
    vif_subset.append((var, vif))

print("VIF para variables correlacionadas:")
for var, vif in vif_subset:
    print(f"{var}: {vif:.2f}")

Paso 3: Criterios de eliminación

Importancia teórica: ¿Cuál es más crítica para el negocio?
Poder predictivo individual: ¿Cuál tiene mayor correlación con Y?
Facilidad de medición: ¿Cuál es más fácil/barata de obtener?

Decisión ejemplo:

In [ ]:
# Eliminar 'experiencia_mesero' porque:
# 1. Información más difícil de obtener
# 2. 'ambiente' y 'precio' son más directos para estrategia de negocio
X_reduced = X_df.drop(['experiencia_mesero'], axis=1)

# Recalcular VIF
X_temp = sm.add_constant(X_reduced)
vif_nuevos = []
for i in range(1, X_temp.shape[1]):  # Excluir constante
    vif = variance_inflation_factor(X_temp.values, i)
    vif_nuevos.append((X_temp.columns[i], vif))

print("VIF después de eliminación:")
for var, vif in vif_nuevos:
    print(f"{var}: {vif:.2f}")

Acción 2: Combinación/Transformación de Variables
Estrategia: Crear variables compuestas
Opción A: Índice de Valor Percibido

In [ ]:
# Combinar precio y ambiente en un índice de "relación calidad-precio"
# Normalizar variables primero
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
precio_norm = scaler.fit_transform(X_df[['precio']])
ambiente_norm = scaler.fit_transform(X_df[['ambiente']])

# Crear índice de valor: ambiente/precio (mayor = mejor valor)
X_df['indice_valor'] = ambiente_norm.flatten() / (precio_norm.flatten() + 0.1)  # +0.1 para evitar división por 0

# Eliminar variables originales problemáticas
X_combinado = X_df.drop(['precio', 'ambiente'], axis=1)

Opción B: Análisis de Componentes Principales (PCA)

In [ ]:
from sklearn.decomposition import PCA

# Aplicar PCA solo a variables correlacionadas
variables_correlacionadas = ['precio', 'ambiente', 'experiencia_mesero']
X_subset = X_df[variables_correlacionadas]

# Normalizar antes de PCA
X_subset_scaled = StandardScaler().fit_transform(X_subset)

# Aplicar PCA
pca = PCA(n_components=2)  # Reducir de 3 a 2 componentes
componentes_pca = pca.fit_transform(X_subset_scaled)

# Crear nuevas variables
X_df['componente_lujo'] = componentes_pca[:, 0]  # Primer componente
X_df['componente_servicio'] = componentes_pca[:, 1]  # Segundo componente

# Verificar varianza explicada
print(f"Varianza explicada por componentes: {pca.explained_variance_ratio_}")
# Ejemplo: [0.68, 0.23] = 91% de varianza original preservada

# Eliminar variables originales
X_pca_final = X_df.drop(variables_correlacionadas, axis=1)

Validación de efectividad:
1. Verificar VIF reducido:

In [ ]:
# Recalcular VIF para modelo transformado
X_final = sm.add_constant(X_combinado)  # o X_pca_final
vif_final = []
for i in range(1, X_final.shape[1]):
    vif = variance_inflation_factor(X_final.values, i)
    vif_final.append((X_final.columns[i], vif))

print("VIF después de combinación:")
for var, vif in vif_final:
    print(f"{var}: {vif:.2f}")

2. Comparar poder predictivo:

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

# Modelo original (con multicolinealidad)
modelo_original = LinearRegression()
scores_original = cross_val_score(modelo_original, X_df, y, cv=5, scoring='r2')

# Modelo reducido
modelo_reducido = LinearRegression()
scores_reducido = cross_val_score(modelo_reducido, X_combinado, y, cv=5, scoring='r2')

print(f"R² promedio original: {scores_original.mean():.4f} (+/- {scores_original.std()*2:.4f})")
print(f"R² promedio reducido: {scores_reducido.mean():.4f} (+/- {scores_reducido.std()*2:.4f})")

Criterios de Selección entre Acciones:
Usar eliminación cuando:

Una variable es claramente redundante
El poder predictivo se mantiene
La interpretabilidad es crítica

Usar combinación cuando:

Todas las variables aportan información única
Se puede crear un índice con significado de negocio
La pérdida de interpretabilidad es aceptable

Resultado esperado: VIF < 5 para todas las variables, manteniendo al menos 85% del poder predictivo original y preservando la interpretabilidad operativa para decisiones de negocio.

## **Parte 6: Variables Categóricas e Interacciones**

1.  Define una **variable categórica** para tu caso (puedes inventarla si no la tenías). Elige una de sus categorías como el nivel **base** o de referencia y **justifica** tu elección.
2.  Crea una **interacción** entre una variable numérica y la variable categórica que definiste. Explica cómo se interpretaría el coeficiente de esta interacción.

Variable Categórica: Tipo de Restaurante
Retomando el caso de Predicción de Satisfacción del Cliente en Restaurantes, voy a definir una variable categórica adicional:
1. Variable Categórica Definida
Variable: X₇ = Tipo de Restaurante
Categorías:

Casual (Fast-casual, cafeterías)
Familiar (Restaurantes de barrio, cadenas familiares)
Fino (Fine dining, alta cocina)

Nivel Base Seleccionado: "Familiar"
Justificación de la elección:
1. Representatividad estadística:

Los restaurantes familiares constituyen aproximadamente 50-60% del mercado
Mayor tamaño de muestra garantiza estimaciones más estables
Reduce la varianza de los coeficientes estimados

2. Punto de referencia lógico:

Representa el "estándar medio" del mercado
Facilita comparaciones: restaurantes casuales vs. familiares, finos vs. familiares
Las expectativas de clientes en restaurantes familiares son más predecibles

3. Interpretabilidad para el negocio:

Los gerentes pueden entender fácilmente desviaciones desde esta base
Permite estrategias de posicionamiento claras (subir hacia "fino" o bajar hacia "casual")

Codificación Dummy resultante:
X₇_Casual = 1 si es Casual, 0 si no
X₇_Fino = 1 si es Fino, 0 si no
(Familiar = 0, 0 - referencia implícita)
Modelo extendido:
Y = β₀ + β₁X₁ + β₂X₂ + β₃X₃ + β₄X₄ + β₅X₅ + β₆X₆ +
    β₇X₇_Casual + β₈X₇_Fino + ε
Interpretación de coeficientes:

β₇: Diferencia promedio en satisfacción entre restaurantes Casuales vs. Familiares
β₈: Diferencia promedio en satisfacción entre restaurantes Finos vs. Familiares


2. Interacción: Variable Numérica × Categórica
Interacción Propuesta: X₃ (Precio) × X₇ (Tipo de Restaurante)
Modelo con interacción:
Y = β₀ + β₁X₁ + β₂X₂ + β₃X₃ + β₄X₄ + β₅X₅ + β₆X₆ +
    β₇X₇_Casual + β₈X₇_Fino +
    β₉(X₃ × X₇_Casual) + β₁₀(X₃ × X₇_Fino) + ε
Términos de interacción:

β₉: X₃ × X₇_Casual (Precio × Dummy_Casual)
β₁₀: X₃ × X₇_Fino (Precio × Dummy_Fino)

Interpretación de los Coeficientes de Interacción
¿Por qué esta interacción es relevante?

"El efecto del precio en la satisfacción del cliente varía según el tipo de restaurante, porque las expectativas y sensibilidad al precio son diferentes en cada segmento."

Interpretación detallada de β₉ (Precio × Casual):
Supongamos β₉ = -0.15

"En restaurantes casuales, por cada dólar adicional en el precio promedio, la satisfacción disminuye 0.15 puntos más de lo que disminuiría en restaurantes familiares."

Desglose matemático:
Para Restaurantes Familiares (base):
Efecto del precio = β₃
Para Restaurantes Casuales:
Efecto del precio = β₃ + β₉ = β₃ + (-0.15)
Ejemplo numérico (β₃ = -0.10):

Familiares: Por cada $1 de aumento, satisfacción baja 0.10 puntos
Casuales: Por cada $1 de aumento, satisfacción baja 0.25 puntos (0.10 + 0.15)

Interpretación detallada de β₁₀ (Precio × Fino):
Supongamos β₁₀ = +0.08

"En restaurantes finos, por cada dólar adicional en el precio promedio, la satisfacción disminuye 0.08 puntos menos de lo que disminuiría en restaurantes familiares."

Para Restaurantes Finos:
Efecto del precio = β₃ + β₁₀ = -0.10 + 0.08 = -0.02
Comparación de sensibilidades al precio:

Casuales: -0.25 puntos por dólar (muy sensibles)
Familiares: -0.10 puntos por dólar (sensibilidad media)
Finos: -0.02 puntos por dólar (poco sensibles)

Interpretación de Negocio
¿Por qué estos coeficientes tienen sentido?
1. Restaurantes Casuales (β₉ = -0.15):

Clientes sensibles al precio: Buscan valor y conveniencia
Expectativas claras: Precio bajo = experiencia aceptable
Elasticidad alta: Aumentos de precio generan insatisfacción desproporcionada

2. Restaurantes Finos (β₁₀ = +0.08):

Clientes menos sensibles: Priorizan calidad sobre precio
Expectativa premium: Precio alto = calidad excepcional esperada
Elasticidad baja: Aumentos de precio no afectan tanto la satisfacción

3. Restaurantes Familiares (base):

Sensibilidad moderada: Balance entre precio y calidad
Expectativas balanceadas: Precio justo por valor recibido

Predicciones Específicas por Segmento
Ejemplo: Plato de $25
Restaurante Casual:
Contribución del precio = β₃ × 25 + β₉ × 25 × 1
                       = -0.10 × 25 + (-0.15) × 25
                       = -2.5 - 3.75 = -6.25 puntos
Restaurante Familiar:
Contribución del precio = β₃ × 25 = -0.10 × 25 = -2.5 puntos
Restaurante Fino:
Contribución del precio = β₃ × 25 + β₁₀ × 25 × 1
                       = -0.10 × 25 + 0.08 × 25
                       = -2.5 + 2.0 = -0.5 puntos
Implicaciones Estratégicas
Para Restaurantes Casuales:

Pricing muy cuidadoso: Pequeños aumentos pueden causar grandes pérdidas de satisfacción
Enfoque en eficiencia: Mantener costos bajos para ofrecer precios atractivos
Valor percibido: Comunicar claramente el valor de la propuesta

Para Restaurantes Finos:

Flexibilidad en pricing: Mayor margen para aumentos de precio
Inversión en calidad: Los clientes toleran precios altos si la calidad es consistente
Premium positioning: Mantener percepción de exclusividad

Para Restaurantes Familiares:

Estrategia balanceada: Equilibrio entre precio competitivo y calidad
Diferenciación: Encontrar nichos que justifiquen precio vs. casuales

Validación del Modelo de Interacción

In [ ]:
import statsmodels.api as sm
import pandas as pd

# Crear variables dummy
df['casual'] = (df['tipo_restaurante'] == 'Casual').astype(int)
df['fino'] = (df['tipo_restaurante'] == 'Fino').astype(int)

# Crear términos de interacción
df['precio_x_casual'] = df['precio'] * df['casual']
df['precio_x_fino'] = df['precio'] * df['fino']

# Ajustar modelo
X = df[['calidad_comida', 'velocidad_servicio', 'precio', 'ambiente',
        'experiencia_mesero', 'dia_semana', 'casual', 'fino',
        'precio_x_casual', 'precio_x_fino']]
X = sm.add_constant(X)
y = df['satisfaccion']

model = sm.OLS(y, X).fit()
print(model.summary())

# Test de significancia de interacciones
f_test = model.f_test(['precio_x_casual = 0', 'precio_x_fino = 0'])
print(f"F-test para interacciones: {f_test}")

Esta interacción revela cómo las diferentes expectativas y comportamientos de clientes según el tipo de restaurante modifican fundamentalmente la relación precio-satisfacción, proporcionando insights accionables para estrategias de pricing diferenciadas por segmento.

## **Parte 7: Conceptos Clave de Clasificación**

Aunque el taller se centra en regresión, estos conceptos son fundamentales en Machine Learning.

1.  Explica qué es la **curva ROC** y para qué se utiliza en un problema de clasificación.

¿Qué es la Curva ROC?
La curva ROC (Receiver Operating Characteristic) es una herramienta gráfica fundamental para evaluar el rendimiento de modelos de clasificación binaria. ROC significa "Característica Operativa del Receptor", un término que proviene de la teoría de detección de señales desarrollada durante la Segunda Guerra Mundial para analizar señales de radar.
Definición y Componentes
La curva ROC es un gráfico bidimensional que representa la relación entre:

Eje Y (Sensibilidad): Tasa de Verdaderos Positivos (TPR = True Positive Rate)
Eje X (1-Especificidad): Tasa de Falsos Positivos (FPR = False Positive Rate)

Fórmulas clave:

TPR (Sensibilidad) = TP / (TP + FN)
FPR (1-Especificidad) = FP / (FP + TN)

Donde:

TP = Verdaderos Positivos
FP = Falsos Positivos
TN = Verdaderos Negativos
FN = Falsos Negativos

¿Para qué se utiliza la Curva ROC?
1. Evaluación de Rendimiento del Modelo
La curva ROC permite visualizar qué tan bien un modelo de clasificación binaria puede distinguir entre las dos clases. Un modelo perfecto tendría una curva que va directamente hacia la esquina superior izquierda (TPR = 1, FPR = 0).
2. Comparación de Modelos
Permite comparar múltiples algoritmos de clasificación en un mismo gráfico. El modelo con la curva más cercana a la esquina superior izquierda generalmente tiene mejor rendimiento.
3. Selección del Punto de Corte Óptimo
Cada punto en la curva ROC representa un umbral de decisión diferente. Esto ayuda a seleccionar el punto de corte que mejor equilibre sensibilidad y especificidad según las necesidades del problema.
4. Cálculo del Área Bajo la Curva (AUC)
El AUC-ROC es una métrica única que resume el rendimiento del modelo:

AUC = 1: Modelo perfecto
AUC = 0.5: Modelo equivalente al azar
AUC < 0.5: Modelo peor que el azar (puede invertirse)

Interpretación Práctica
Formas típicas de la curva:
Curva ideal: Se acerca a la esquina superior izquierda, indicando alta sensibilidad con baja tasa de falsos positivos.
Línea diagonal: Representa un clasificador aleatorio (AUC = 0.5), sin capacidad discriminativa.
Curva por debajo de la diagonal: Indica un modelo que está haciendo predicciones inversas a las correctas.
Ventajas de la Curva ROC

Independent del umbral: Evalúa el modelo en todos los posibles puntos de corte
Invariante a la escala: No se ve afectada por la escala de las probabilidades predichas
Comparación visual clara: Permite comparar fácilmente múltiples modelos
Métrica única (AUC): Proporciona un valor numérico para comparaciones objetivas

Limitaciones Importantes

Problemas con datasets desbalanceados: En casos de clases muy desproporcionadas, la curva ROC puede ser demasiado optimista
No considera costos: No tiene en cuenta si los falsos positivos o falsos negativos tienen costos diferentes
Solo para clasificación binaria: Requiere adaptaciones para problemas multiclase

Cuándo Usar ROC vs Otras Métricas
Usar ROC cuando:

Las clases están relativamente balanceadas
Los costos de FP y FN son similares
Se necesita evaluar el modelo en diferentes umbrales

Considerar alternativas cuando:

Hay un gran desbalance de clases (usar curva Precision-Recall)
Los costos de errores son muy diferentes
Se necesita una métrica más interpretable para stakeholders no técnicos


2.  Define el concepto de **accuracy** (exactitud) y menciona una situación en la que podría ser una métrica engañosa.
¿Qué es la Accuracy (Exactitud)?
La accuracy o exactitud es la métrica más básica e intuitiva para evaluar modelos de clasificación. Mide la proporción de predicciones correctas que hace el modelo sobre el total de predicciones realizadas.
Fórmula:
Accuracy = (TP + TN) / (TP + TN + FP + FN)
Donde:

TP = Verdaderos Positivos (predicciones correctas de la clase positiva)
TN = Verdaderos Negativos (predicciones correctas de la clase negativa)
FP = Falsos Positivos (predicciones incorrectas como positivo)
FN = Falsos Negativos (predicciones incorrectas como negativo)

Interpretación:

Valor: Entre 0 y 1 (o 0% y 100%)
Interpretación: Porcentaje de casos clasificados correctamente
Ejemplo: Accuracy = 0.85 significa que el modelo acierta en el 85% de los casos

¿Cuándo la Accuracy Puede Ser Engañosa?
Situación Crítica: Datasets Desbalanceados
La accuracy se vuelve una métrica profundamente engañosa cuando trabajamos con clases desbalanceadas, donde una clase es mucho más frecuente que la otra.
Ejemplo Práctico: Detección de Fraude Bancario
Imagina un modelo para detectar transacciones fraudulentas con las siguientes características:
Dataset:

Transacciones legítimas: 9,950 casos (99.5%)
Transacciones fraudulentas: 50 casos (0.5%)
Total: 10,000 transacciones

Modelo "Ingenuo":
Un modelo que simplemente predice "NO FRAUDE" para todos los casos tendría:

Predicciones correctas: 9,950 (todas las legítimas)
Predicciones incorrectas: 50 (todos los fraudes no detectados)
Accuracy = 9,950/10,000 = 99.5%

¿Por qué es Engañosa esta Accuracy?
Aparenta excelente rendimiento: Un 99.5% de accuracy suena impresionante y podría llevar a creer que tenemos un modelo excelente.
Realidad crítica: El modelo es completamente inútil porque:

No detecta ningún fraude: 0% de detección en fraudes reales
Costo empresarial enorme: Todas las transacciones fraudulentas pasan desapercibidas
Falla en su propósito principal: El objetivo era detectar fraudes, no clasificar transacciones legítimas

Otros Escenarios Donde la Accuracy Engaña
1. Diagnóstico de Enfermedades Raras

Dataset: 99% pacientes sanos, 1% con enfermedad rara
Modelo que siempre dice "sano": 99% accuracy
Problema: 0% de detección de la enfermedad que necesitamos diagnosticar

2. Detección de Spam

Dataset: 95% emails legítimos, 5% spam
Modelo que nunca detecta spam: 95% accuracy
Problema: Toda la bandeja se llena de spam

3. Control de Calidad Industrial

Dataset: 98% productos buenos, 2% defectuosos
Modelo que aprueba todo: 98% accuracy
Problema: Productos defectuosos llegan al cliente

¿Por Qué Ocurre Este Problema?
Dominancia de la Clase Mayoritaria
La accuracy se ve dominada por la clase más frecuente. En datasets desbalanceados, acertar en la clase mayoritaria contribuye desproporcionalmente al resultado final.
Insensibilidad al Propósito del Modelo
La accuracy no distingue entre la importancia de detectar correctamente diferentes clases. En muchos casos prácticos, detectar la clase minoritaria es más crítico.
Métricas Alternativas Más Apropiadas
Para Datasets Desbalanceados:
1. Precision (Precisión):

Precision = TP / (TP + FP)
¿De las predicciones positivas, cuántas son correctas?

2. Recall (Sensibilidad):

Recall = TP / (TP + FN)
¿De los casos positivos reales, cuántos detectamos?

3. F1-Score:

F1 = 2 × (Precision × Recall) / (Precision + Recall)
Media armónica entre precision y recall

4. AUC-ROC:

Área bajo la curva ROC
Evalúa el rendimiento en todos los umbrales

5. Balanced Accuracy:

(Sensibilidad + Especificidad) / 2
Promedia el rendimiento en ambas clases

¿Cuándo Sí Usar Accuracy?
La accuracy sigue siendo útil cuando:

Clases balanceadas: Distribución similar entre clases
Costos equivalentes: El costo de FP y FN es similar
Objetivo general: Se busca rendimiento global sin priorizar ninguna clase
Comparación inicial: Como primera aproximación para comparar modelos

Recomendación Práctica
Nunca uses accuracy como única métrica. Siempre complementa con:

Análisis de la distribución de clases
Matriz de confusión para ver errores específicos
Métricas específicas según el problema (precision, recall, F1)
Consideración del contexto de negocio y costos de errores

La accuracy del 99.5% en el ejemplo de fraudes puede costar millones de dólares en pérdidas reales, mientras que un modelo con 85% accuracy pero que detecta 80% de los fraudes sería infinitamente más valioso. El contexto y propósito del modelo siempre deben guiar la selección de métricas de evaluación.

3.  Describe qué es una **matriz de confusión** y cómo se interpretan sus componentes (Verdaderos Positivos, Falsos Positivos, Verdaderos Negativos, Falsos Negativos).
¿Qué es una Matriz de Confusión?
La matriz de confusión es una tabla de contingencia que proporciona una representación visual detallada del rendimiento de un modelo de clasificación. Es una herramienta fundamental que desglosa todas las predicciones del modelo comparándolas con los valores reales, permitiendo identificar exactamente dónde y cómo se está "confundiendo" el algoritmo.
Estructura Básica (Clasificación Binaria)
                    PREDICCIÓN
                 Positivo  Negativo
REAL Positivo      TP       FN
     Negativo      FP       TN
Donde:

Filas: Clases reales (ground truth)
Columnas: Predicciones del modelo
Celdas: Cantidad de casos en cada combinación

Componentes de la Matriz de Confusión
1. Verdaderos Positivos (TP - True Positives)
Definición: Casos donde el modelo predijo correctamente la clase positiva.

Real: Positivo
Predicción: Positivo
Resultado:  CORRECTO

Ejemplo práctico: En diagnóstico médico, pacientes que realmente tienen la enfermedad Y el modelo los diagnosticó como enfermos.
2. Falsos Positivos (FP - False Positives)
Definición: Casos donde el modelo predijo incorrectamente la clase positiva.

Real: Negativo
Predicción: Positivo
Resultado:  ERROR (Falsa Alarma)

Ejemplo práctico: Pacientes sanos que el modelo diagnosticó incorrectamente como enfermos. También llamado Error Tipo I.
3. Verdaderos Negativos (TN - True Negatives)
Definición: Casos donde el modelo predijo correctamente la clase negativa.

Real: Negativo
Predicción: Negativo
Resultado:  CORRECTO

Ejemplo práctico: Pacientes sanos que el modelo correctamente diagnosticó como sanos.
4. Falsos Negativos (FN - False Negatives)
Definición: Casos donde el modelo predijo incorrectamente la clase negativa.

Real: Positivo
Predicción: Negativo
Resultado:  ERROR (Caso Perdido)

Ejemplo práctico: Pacientes enfermos que el modelo no logró detectar, diagnosticándolos como sanos. También llamado Error Tipo II.